# 목차
<hr/>

<ol>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">구글 드라이브 mount하여 연동하기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">필요한 모듈 불러오기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 데이터 읽어오기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : 데이터 정제</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 3</span> : 데이터를 Tensor로 변환하기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 4</span> : 평가 데이터셋 분리</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> 생성된 텐서를 소스와 타겟으로 분리하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 훈련 데이터와 평가 데이터를 분리하기</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 인공지능 만들기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> loss 지정하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 텍스트 생성 클래스 선언하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">3. </span> Model Build</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">4. </span> Model 훈련하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">5. </span> 작문을 통해 모델 평가해보기</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">회고</i>
</ol>

<div style = "height: 50px;"><div/>

# 구글 드라이브 mount하여 연동하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<div style = "height: 50px;"><div/>

# 필요한 모듈 불러오기

In [89]:
import warnings
warnings.filterwarnings("ignore")

import glob
import os, re

import tensorflow        as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 데이터 읽어오기

In [16]:
txt_file_path = "/content/drive/MyDrive/data/*"
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("\n데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


데이터 크기: 187088
Examples:
 ['Jolene, Jolene, Jolene, Jolene', "I'm begging of you please don't take my man", 'Jolene, Jolene, Jolene, Jolene']


<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : 데이터 정제

<p style= "font-size:15px; line-height:24px;"> - preprocess_sentence() 사용하여 데이터 정제하기</p>

### 함수

In [17]:
# 문장 필터링하는 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

<div style = "height: 25px;"></div>

### 데이터 정제하기

In [18]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: 
        continue
    
    if sentence[-1] == ":": 
        continue
    
    # 토큰의 개수 15개 초과 시, 삭제하기
    if len(sentence.split(' ')) >17: # <start>, <end> 포함하여 17토큰으로 값 설정
        continue
        
    # 정제를 하고 담아주세요tokenizer.num_words
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

<div style = "height: 25px;"></div>

### 정제된 데이터 확인하기

In [19]:
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> jolene , jolene , jolene , jolene <end>',
 '<start> i m begging of you please don t take my man <end>',
 '<start> jolene , jolene , jolene , jolene <end>',
 '<start> please don t take him just because you can your beauty is beyond compare <end>',
 '<start> with flaming locks of auburn hair <end>',
 '<start> with ivory skin and eyes of emerald green <end>',
 '<start> your smile is like a breath of spring <end>',
 '<start> your voice is soft like summer rain <end>',
 '<start> and i cannot compete with you <end>',
 '<start> jolene he talks about you in his sleep <end>']

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 3</span> : 데이터를 Tensor로 변환하기

### 함수

In [20]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000, 
    filters=' ',
    oov_token="<unk>")
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

<div style = "height: 25px;"></div>

### Tensor 변환하기

In [21]:
tensor, tokenizer = tokenize(corpus)

[[   2 1444    4 ...    0    0    0]
 [   2    5   22 ...    0    0    0]
 [   2 1444    4 ...    0    0    0]
 ...
 [   2    7  456 ...    0    0    0]
 [   2   31  122 ...    0    0    0]
 [   2    5  352 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe01f286610>


In [22]:
print(tensor[:3, :10])

[[   2 1444    4 1444    4 1444    4 1444    3    0]
 [   2    5   22 2279   19    7  213   36   15   82]
 [   2 1444    4 1444    4 1444    4 1444    3    0]]


<div style = "height: 25px;"></div>

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 4</span> : 평가 데이터셋 분리

## 1. 생성된 텐서를 소스와 타겟으로 분리하기
<hr/>

In [23]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1] 

# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 1444    4 1444    4 1444    4 1444    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[1444    4 1444    4 1444    4 1444    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


<div style = "height: 25px;"></div>

### 분리된 데이터 확인해보기

In [24]:
src_input

array([[   2, 1444,    4, ...,    0,    0,    0],
       [   2,    5,   22, ...,    0,    0,    0],
       [   2, 1444,    4, ...,    0,    0,    0],
       ...,
       [   2,    7,  456, ...,    0,    0,    0],
       [   2,   31,  122, ...,    0,    0,    0],
       [   2,    5,  352, ...,    0,    0,    0]], dtype=int32)

<div style = "height: 50px;"><div/>

## 2. 훈련 데이터와 평가 데이터를 분리하기
<hr/>

### train_test_split() 함수 사용하여 훈련 데이터와 검증 데이터 분리하기

In [25]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)

<div style = "height: 25px;"></div>

### from_tensor_slices() 메소드를 이용해 훈련 데이터 생성하기

In [27]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 512
steps_per_epoch = len(enc_train) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   

In [28]:
# from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(512, 32), dtype=tf.int32, name=None), TensorSpec(shape=(512, 32), dtype=tf.int32, name=None))>

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 인공지능 만들기

## 1. loss 지정하기
<hr/>

In [29]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

<div style = "height: 50px;"><div/>

## 2. 텍스트 생성 클래스 선언하기
<hr/>

In [61]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

<div style = "height: 50px;"><div/>

## 3. Model Build
<hr/>

In [33]:
embedding_size = 512
hidden_size = 1290
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [34]:
model.summary()

ValueError: ignored

<p style= "font-size:15px; line-height:24px;">모델이 빌드되지 않은 상태에서 summary를 진행했기 때문에, 모델이 무슨 일을 하는 모델인지 알려주지 못한다. 따라서, 모델을 build하기 위해서는 어떤 데이터를 입력받아야하는지 알려줘야 하므로 데이터를(소스 데이터 = 특성값 = x값)을 모델에 넣어야한다.</p>

<div style = "height: 25px;"><div/>

### 데이터를 모델에 넣어주기

In [35]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(512, 32, 12001), dtype=float32, numpy=
array([[[-8.33122831e-05,  3.46817134e-04, -3.92990740e-04, ...,
         -8.72974997e-05, -1.18939512e-04,  1.76366040e-04],
        [-2.13843450e-04,  6.09053357e-04, -5.01786184e-04, ...,
         -2.71189114e-04, -1.30620057e-04,  7.58145179e-05],
        [-3.86614702e-04,  6.02878747e-04, -2.29267098e-04, ...,
         -4.35470254e-04,  1.05836538e-04,  1.25871913e-04],
        ...,
        [-2.31503556e-03,  4.32351138e-04,  3.30807082e-03, ...,
          2.73769628e-03, -1.17186806e-03, -1.25766860e-03],
        [-2.31074565e-03,  4.28909465e-04,  3.30926012e-03, ...,
          2.74513150e-03, -1.18634116e-03, -1.26745214e-03],
        [-2.30704714e-03,  4.25611099e-04,  3.31096072e-03, ...,
          2.75090220e-03, -1.19897421e-03, -1.27607968e-03]],

       [[-8.33122831e-05,  3.46817134e-04, -3.92990740e-04, ...,
         -8.72974997e-05, -1.18939512e-04,  1.76366040e-04],
        [-3.12965974e-04,  4.36402799e-04, -8

<div style = "height: 25px;"><div/>

### 모델의 정보 확인하기

In [36]:
# 이제 모델이 뭐하는지 알았으니 summary해서 모델의 정보 확인하기
model.summary()

Model: "text_generator_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  6144512   
                                                                 
 lstm_4 (LSTM)               multiple                  9303480   
                                                                 
 lstm_5 (LSTM)               multiple                  13317960  
                                                                 
 dense_2 (Dense)             multiple                  15493291  
                                                                 
Total params: 44,259,243
Trainable params: 44,259,243
Non-trainable params: 0
_________________________________________________________________


<p style= "font-size:15px; line-height:24px;">모델의 파라미터가 44,259,243이 나왔다.<br/>
따라서, 모델을 돌리는데에 있어 <span style = 'background-color:#fff5b1; padding:0.2px;'>시간이 오래 걸릴 것</span>이라고 예상된다.<br/>
뿐만아니라, 파라미터가 많기 때문에 <span style = 'background-color:#fff5b1; padding:0.2px;'>과적합</span>이 발생할 수 있다는 문제가 발생할 수 있다고 생각한다.</p>

<div style = "height: 50px;"><div/>

## 4. Model 훈련하기
<hr/>

In [37]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset, validation_data = (enc_val, dec_val), epochs = 10)

Epoch 1/10
267/267 [==============================] - 458s 2s/step - loss: 1.9041 - val_loss: 1.5502
Epoch 2/10
267/267 [==============================] - 473s 2s/step - loss: 1.4691 - val_loss: 1.4186
Epoch 3/10
267/267 [==============================] - 472s 2s/step - loss: 1.3720 - val_loss: 1.3518
Epoch 4/10
267/267 [==============================] - 473s 2s/step - loss: 1.2990 - val_loss: 1.3005
Epoch 5/10
267/267 [==============================] - 473s 2s/step - loss: 1.2866 - val_loss: 1.2745
Epoch 6/10
267/267 [==============================] - 472s 2s/step - loss: 1.1916 - val_loss: 1.2396
Epoch 7/10
267/267 [==============================] - 469s 2s/step - loss: 1.1356 - val_loss: 1.2103
Epoch 8/10
267/267 [==============================] - 471s 2s/step - loss: 1.0822 - val_loss: 1.1858
Epoch 9/10
267/267 [==============================] - 472s 2s/step - loss: 1.0313 - val_loss: 1.1639
Epoch 10/10
267/267 [==============================] - 473s 2s/step - loss: 0.9828 - val_lo

<p style= "font-size:15px; line-height:24px;">모델을 훈련시킨 시간은 <span style = 'background-color:#fff5b1; padding:0.2px;'>총 2시간 30분</span>이 걸렸다.<br/>
예상한대로, 히든레이어와, 임베딩 사이즈를 크게 주었기 때문에 파라미터가 많이 생성되었고, 이로 인해 모델의 훈련 속도가 느려진 것 같다.
loss값은 과적합 되었을 것이라는 예상과는 다르게, 훈련 데이터의 loss값과 검증 데이터셋의 loss값이 크게 차이가 나지 않으면서, <span style = 'background-color:#fff5b1; padding:0.2px;'>2.2 이하로 떨어졌다.</span></p>

<div style = "height: 50px;"><div/>

## 5. 작문을 통해 모델 평가해보기
<hr/>

### 작문 진행 함수 선언하기

In [38]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

<div style = "height: 50px;"><div/>

### 작문해보기

In [39]:
generate_text(model, tokenizer, init_sentence="<start> always", max_len=20)

'<start> always be a little selfish <end> '

In [40]:
generate_text(model, tokenizer, init_sentence="<start> i think", max_len=20)

'<start> i think i m not sounding too desperate <end> '

In [42]:
generate_text(model, tokenizer, init_sentence="<start> i want", max_len=20)

'<start> i want to sunroof top <end> '

In [43]:
generate_text(model, tokenizer, init_sentence="<start> please", max_len=20)

'<start> please don t stop the music <end> '

In [44]:
generate_text(model, tokenizer, init_sentence="<start> from", max_len=20)

'<start> from the bottom of my broken heart <end> '

In [45]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a beautiful thing <end> '

In [46]:
generate_text(model, tokenizer, init_sentence="<start> I love", max_len=20)

'<start> i love you , i love you , i need you <end> '

<p style= "font-size:15px; line-height:24px;">대부분 잘 나온 것을 확인할 수 있었으나, 해석해보니 소위 중2병 같은 내용이 꽤 많았다.<br/>
또한, 출력한 문장 중 sunroof top이 무슨 뜻인지 모르겠다.<br/>
검색해본 결과, Pacewon - Sunroof Top이라는 노래가 있는 것을 확인하였는데 뜻은 찾을 수 없었다.</p>

<div style = "height: 50px;"><div/>

# 회고
<hr/>

<p style= "font-size:16px; line-height:30px;">이번 프로젝트를 하면서 처음으로 코랩 pro를 사용해보았는데, 평소 사용하던 jupyter notebook과는 사용 방법이 많이 달라 사용하는데 애를 많이 먹었습니다.</p>

<div style = "height: 50px;"></div>

## 1. 코랩에서 파일을 불러오기 위해서는 구글 드라이브에 올려 불러오는 것이 편하다.
<hr/>

<p style= "font-size:16px; line-height:30px;">로컬 파일을 불러오고자 경로 설정을 하였는데 되지 않아 확인해본 결과,</p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<p style= "font-size:16px; line-height:30px;">를 통해 구글 드라이브와 연동을 시킨 후 파일을 불러올 수 있었다.<br/>
또한, 이때 기본 주소(HOME)는 content/가 되며, 이는 os.getcwd()를 통해 현재 경로를 확인할 수 있다.</p>

<div style = "height: 50px;"></div>

## 2. 드라이브 상의 파일의 경로 확인하기
<hr/>

<img src = "img/colab_01.png">

<p style= "font-size:16px; line-height:30px;">왼쪽 파일 아이콘을 눌러 폴더 목록을 열은 후 원하는 파일에 오른쪽 클릭을 하여 경로를 복사할 수 있다.</p>

<div style = "height: 50px;"></div>

## 3. 모델을 fit할 때에는 꼭 변수를 지정하자
<hr/>

<p style= "font-size:16px; line-height:30px;">모델의 훈련 과정과 데이터의 loss값 등을 이용하기 위해서는 model.fit()할 때, </p>

In [ ]:
history = model.fit()

<p style= "font-size:16px; line-height:30px;">이렇게 변수를 지정하여 값을 저장해야, 후에 loss값 비교, 모델별 성능 비교 등을 할 수 있으므로, <span style = 'background-color:#fff5b1; padding:0.2px;'>반드시 변수를 생성하는 것이 좋다.</span> <br/>
모델을 모두 학습시키고 난 후, 이 사실을 알게되어서 매우 슬펐다. <br/>
이번에 모델을 돌릴 때에는 시간이 많이 소요됐으므로 넘어가고, 다음에는 반드시 변수에 저장하여 사용할 것이다.</p>

<div style = "height: 50px;"></div>

## 4. 서브 클래스를 사용하여 모델을 설계했을 때에는 tf.keras.callbacks.ModelCheckpoint로 저장하자
<hr/>

- [콜백 사용하는 방법 1](https://deep-deep-deep.tistory.com/53)
- [콜백 사용하는 방법 2](https://teddylee777.github.io/tensorflow/keras-%EC%BD%9C%EB%B0%B1%ED%95%A8%EC%88%98-vol-02)
- [텐서플로우 콜백](https://www.tensorflow.org/guide/checkpoint?hl=ko)
- [model.save()](https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ko)

<p style= "font-size:16px; line-height:30px;">파일을 닫고 다시 열었을 때에는 모델을 다시 사용할 수 없으므로, model.save()를 사용하여 .h5(h5파일)로 저장해야한다. 이 후, tf.saved_model.load()로 쉽게 불러올 수 있다.<br/>
<span style = 'background-color:#fff5b1; padding:0.2px;'>그러나, 이 방법은 함수형 또는 시퀀셜 API(모델을 순차적으로 쌓음)를 사용하여 만든 모델에만 사용하 가능하다.</span></p>

<p style= "font-size:16px; line-height:30px;">실제로 모델을 돌렸을 시,  save_format='tf'으로 지정하라는 오류가 발생하는 것을 확인할 수 있었다.</p>

<p style= "font-size:16px; line-height:30px;">이 후, 모델을 tf로 저장하여 불러오니, model.summary()는 먹히나, 모델 입력 값의 shape가 다르다는 오류가 발생하였다.
확인해본 결과, model.save()는 서브클래스를 사용한 모델은 저장할 수 없으며, 가중치만 저장하여 사용할 수 있다는 것을 확인할 수 있었다. <br/>
때문에, 파일로 저장된 모델과 실제 적용하는 모델이 달라져서 적용할 수 없었다.</p>

<p style= "font-size:16px; line-height:30px;">서브 클래스를 사용하여 모델을 설계했다면, <span style = 'background-color:#fff5b1; padding:0.2px;'>tf.keras.callbacks.ModelCheckpoint</span>로 모델의 저장하여 불러올 수 있다.<br/>
ModelCheckpoint는 모델이 학습하면서 정의한 조건(fit())을 만족했을 때, Model의 가중치 값을 중간중간 저장한다. 학습시간이 오래 걸린다면 loss값이 올라갔을 때에는 저장하지 않고, 떨어졌을 때에는 가중치를 저장함으로서, 중간에 과적합이 발생하였거나, 다양한 문제가 발생했을 때, 다시 중간의 가중치 값을 불러와서 학습을 이어나갈 수 있다.</p>

<div style = "height: 50px;"></div>

## 5. 코랩은 모델을 fit()하는 도중, 시간이 지나면 런타임이 끊긴다.  
<hr/>

<p style= "font-size:16px; line-height:30px;">이번에는 모델의 훈련 시간이 오래 걸리기 때문에, 자기 직전 model.fit()을 실행시켰다.
자고 일어나면 모델 훈련이 끝났을 것이라는 기대와는 달리, 중간에 런타임이 끊겨 모델을 다시 사용할 수도 없고, 훈련 도중에 런타임이 끊긴 것을 확인할 수 있었다.<br/>
조심하여 돌려야겠다.</p>